# 参数管理

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.4334],
        [-0.4698]], grad_fn=<AddmmBackward0>)

## 参数访问

### 访问某层的参数

In [2]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2451, -0.1810,  0.2240,  0.1733,  0.3161, -0.2581,  0.2942,  0.3455]])), ('bias', tensor([-0.1788]))])


这里 `net[2]` 表示第 3 个神经网络层，也就是 `nn.Linear(8, 1)`。还可以用 `bias`、`weight` 等方法来访问偏置和权重：

In [9]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
print()
print(type(net[2].weight))
print(net[2].weight)
print(net[2].weight.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1788], requires_grad=True)
tensor([-0.1788])

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([[ 0.2451, -0.1810,  0.2240,  0.1733,  0.3161, -0.2581,  0.2942,  0.3455]],
       requires_grad=True)
tensor([[ 0.2451, -0.1810,  0.2240,  0.1733,  0.3161, -0.2581,  0.2942,  0.3455]])


### 一次性访问所有参数

In [10]:
print(*[(name, param.shape) for name, param in net.named_parameters()])
print(*[(name, param.shape) for name, param in net[2].named_parameters()])

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))
('weight', torch.Size([1, 8])) ('bias', torch.Size([1]))


### 嵌套块参数访问

先定义一个嵌套块：

In [11]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.2606],
        [0.2606]], grad_fn=<AddmmBackward0>)

打印嵌套块结构：

In [12]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


使用索引访问。例如访问第一个主要的块中、第二个子块的第一层的偏置项。

In [13]:
print(rgnet[0][1][0].bias)

Parameter containing:
tensor([ 0.3133,  0.4230, -0.1578,  0.2380,  0.4215,  0.1318, -0.1221, -0.0255],
       requires_grad=True)


## 参数初始化

使用 `.apply()` 方法：

In [14]:
def init_normal(m):
    # 对所有线性层进行初始化
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0128, -0.0006, -0.0047, -0.0048]), tensor(0.))

既可以调用 `torch.nn.init` 中的方法，常用的 `torch.nn.init` 中的方法有：

- `normal_`: Gaussian 分布
- `uniform_`: 平均分布
- `zeros_`: 零初始化
- `constant_`: 初始化为常量
- `xavier_normal_`, `xavier_uniform_`
- `kaiming_normal_`, `kaiming_uniform_`

也可以自行编写初始化代码，例如：

In [ ]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

`.apply` 方法也可以专门针对某一层使用，例如：

In [ ]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

也可以在 `__init__` 方法中就进行初始化，直接调用 `nn.init.xxx` 方法

## 参数绑定

可以通过在 `nn.Sequential` 参数中使用相同的变量来达到参数绑定的效果。

此时在进行反向传播时，该共享参数会被多次更新。可以通过关闭某一处的 `grad` 来只进行一次更新

In [ ]:
shared_layer = nn.Linear(20, 128)
net = nn.Sequential(shared_layer, nn.ReLU(), 
                    nn.Linear(128, 20), nn.ReLU(),
                    shared_layer, nn.ReLU(),
                    nn.Linear(128, 10))